In [1]:
import tensorflow as tf
import numpy as np
import math
import sys
import os
print('Done.')

Done.


In [7]:
# Input: parametrized heights
# Output: volume

def placeholder_inputs(batch_size, num_param):
    param_pl = tf.placeholder(name="input_param", dtype=tf.float32, shape=(batch_size, num_param))
    vol_pl = tf.placeholder(name="input_vol", dtype=tf.float32, shape=(batch_size))
    
    return param_pl, vol_pl

def load_input(train_path):
    train = open(train_path, 'r')
    train_x = []
    train_y = []
    for line in train:
        data = eval(line)
        if data[1] > 0:
            train_x.append(data[0])
            inv_vol = 1/data[1]
            train_y.append(inv_vol)
    print ('Number of data: ', len(train_x))
    
    return train_x, train_y

def fl_linear(param, batch_size, num_param):
    with tf.variable_scope("fl_linear", reuse=True):
        input_param = tf.placeholder(name="input_param", dtype=tf.float32, shape=(batch_size, num_param))
        input_vol = tf.placeholder(name="input_vol", dtype=tf.float32, shape=(batch_size))
        # Linear activation
        net = tf.fully_connected(1, activation_fn=None, scope='fc1')
    
    return net

def mat_mul(param, batch_size, num_param):
    with tf.variable_scope("mat_mul", reuse=True):
        input_parameter = tf.placeholder(name="input_parameter", dtype=tf.float32, shape=(batch_size, num_param))
        input_volume = tf.placeholder(name="input_volume", dtype=tf.float32, shape=(batch_size))
        W = tf.Variable(tf.random_normal([num_param, 1], dtype=tf.float32), name="W")
        b = tf.Variable(tf.zeros([batch_size, 1], dtype=tf.float32), name="b")
        y = tf.add(tf.matmul(input_parameter, W), b)
        cost_op = tf.reduce_mean(tf.pow(y-input_volume, 2))

    return y, cost_op, input_parameter, input_volume

def train(data, vol):
    
    input_data = data[0:1000]
    input_vol = vol[0:1000]
    
    test_data = data[1000:2000]
    test_vol = vol[1000:2000]
    
    num_data = len(input_data)
    num_param = len(input_data[0])
    num_test = len(test_data)
    print('number of data: ', num_data)
    print('number of parameters: ', num_param)
    print('number of test: ', num_test)
    
    num_batches = int(num_data/batch_size)
    test_batches = int(num_test/batch_size)
    input_param = np.array(input_data)
    input_vol = np.array(input_vol)
    test_param = np.array(test_data)
    test_vol = np.array(test_vol)
    
    #y = fl_linear(input_param)
    y, cost_op, input_parameter, input_volume = mat_mul(input_param, batch_size, num_param)
    
    learning_rate = tf.Variable(0.5, trainable=False)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(cost_op)
    
    sess = tf.Session() # Create TensorFlow session
    print ("Beginning Training")
    with sess.as_default():
        init = tf.global_variables_initializer()
        sess.run(init)
        sess.run(tf.assign(learning_rate, alpha))
        for epoch in range(1, max_epochs):
            print('epoch: ', epoch)
            # Train one epoch
            train_one_epoch(sess, cost_op, train_op, num_batches, input_parameter, input_param, input_volume, input_vol)
            eval_one_epoch(sess, cost_op, test_batches, input_parameter, test_param, input_volume, test_vol)
            
        print('Done.')
        W = tf.get_variable("W")
        print("Weight: ", W.eval())
            

def train_one_epoch(sess, cost_op, train_op, num_batches, input_parameter, input_param, input_volume, input_vol):
     for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        cost,_ = sess.run([cost_op, train_op], feed_dict={input_parameter: input_param[start_idx:end_idx], input_volume: input_vol[start_idx:end_idx]})
        print("loss: ", cost)
        
def eval_one_epoch(sess, cost_op, test_batches, input_parameter, test_param, input_volume, test_vol):
    for batch_idx in range(test_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        cost = sess.run(cost_op, feed_dict={input_parameter: test_param[start_idx:end_idx], input_volume: test_vol[start_idx:end_idx]})
        print("test loss: ", cost)

# Parameters:
last_cost = 0
alpha = 0.4
batch_size = 10
max_epochs = 100
tolerance = 1e-3

    
train_path = '/home/carnd/CYML/output/train/cylinder/tri_1_to_50_2.txt'
train_data, train_vol = load_input(train_path)
print ('sample data: ', train_data[0])
train(train_data, train_vol)

Number of data:  10100
sample data:  [0, 1, 1]
number of data:  1000
number of parameters:  3
number of test:  1000
Beginning Training
epoch:  1
loss:  3.1031156
loss:  7.9520044
loss:  9.177099
loss:  12.003052
loss:  14.035508
loss:  8.885265
loss:  9.026624
loss:  4.992687
loss:  9.242185
loss:  26.32302
loss:  1.9487853
loss:  6.0147943
loss:  23.573664
loss:  3.2422874
loss:  7.05889
loss:  4.739183
loss:  10.82374
loss:  3.471589
loss:  7.0185413
loss:  6.301442
loss:  25.917585
loss:  2.4931765
loss:  5.97263
loss:  15.561093
loss:  12.069449
loss:  20.573374
loss:  3.9292612
loss:  6.4307714
loss:  8.255787
loss:  4.9081564
loss:  38.885517
loss:  1.6573607
loss:  2.7661383
loss:  11.351982
loss:  18.123781
loss:  14.559143
loss:  1.4899789
loss:  42.973263
loss:  7.304552
loss:  10.019865
loss:  8.877336
loss:  3.2558935
loss:  0.5282096
loss:  5.084958
loss:  53.775986
loss:  1.5594122
loss:  5.7328963
loss:  11.339006
loss:  9.561106
loss:  4.8412266
loss:  1.1080605
loss:  

NameError: name 'test_param' is not defined

In [9]:
data = np.arange(6).reshape(2,3)
param = np.array([1,1,1])
input_data = tf.placeholder(name="input_data", dtype=tf.float32, shape=(2, 3))
input_param = tf.placeholder(name="input_param", dtype=tf.float32, shape=(1,3))
for i in range(2):
    row = input_data[i,]
    print(row)
    tmp = tf.concat([row, input_param], 1)
    try:
        out = tf.stack([input_param, tmp])
    except:
        out = tmp
        
sess = tf.Session() # Create TensorFlow session
saver = tf.train.Saver()
with sess.as_default():
    init = tf.global_variables_initializer()
    sess.run(init)
    print("slice: ", sess.run(row, feed_dict = {input_data:data, input_param:param}))
    print("stack result: ", sess.run(out, feed_dict = {input_data:data, input_param:param}))

Tensor("strided_slice_5:0", shape=(3,), dtype=float32)


ValueError: Shape must be rank 1 but is rank 2 for 'concat_5' (op: 'ConcatV2') with input shapes: [3], [1,3], [].

In [15]:
sess = tf.Session()
x = tf.constant([[1,2,3],[8,9,10]])
print(x)
#y = tf.transpose(x[0,:])
x_0 = x[0,:]
x_0 = tf.reshape(x_0,[1,-1])
x_1 = x[1,:]
x_1 = tf.reshape(x_1,[1,-1])
print(x_0)
print(x_0.eval(session=sess).tolist())
z = tf.concat([x_0,x_1],1)
print(z)
print(z.eval(session=sess).tolist())
mul = tf.matmul(tf.transpose(x_0), x_0)
print(mul)
print(mul.eval(session=sess))
for i in range(3):
    concat = tf.reshape(mul[i,i:],[1,-1])
    mul_tmp = tf.reshape(mul[i,i:],[1,-1])
    concat = tf.concat([])

Tensor("Const_12:0", shape=(2, 3), dtype=int32)
Tensor("Reshape_14:0", shape=(1, 3), dtype=int32)
[[1, 2, 3]]
Tensor("concat_12:0", shape=(1, 6), dtype=int32)
[[1, 2, 3, 8, 9, 10]]
Tensor("MatMul_4:0", shape=(3, 3), dtype=int32)
[[1 2 3]
 [2 4 6]
 [3 6 9]]


In [4]:
import tensorflow as tf
import numpy as np
import math
import sys
import os
print('Done.')

def expand_2(input_param_1, batch_size, num_param):
    for row in range(batch_size):
        param_tmp = input_param_1[row,:]
        param_tmp = tf.reshape(param_tmp, [1, -1])
        param_mat = tf.matmul(tf.transpose(param_tmp),param_tmp)

        for i in range(num_param):
            param_tmp2 = tf.reshape(param_mat[i,i:], [1,-1])
            #print(param_tmp2.eval(session=sess))
            param_tmp = tf.concat([param_tmp,param_tmp2], 1)
        param_tmp = tf.reshape(param_tmp, [-1])

        try:
            input_parameter = tf.concat([input_parameter,param_tmp[None, :]],0)
            input_parameter = tf.reshape(input_parameter, [row+1, -1])
        except:
            input_parameter = param_tmp
            input_parameter = tf.reshape(input_parameter, [row+1, -1])
    
    return input_parameter

sess = tf.Session()
data = np.arange(18)
data = data.reshape([6,-1])
batch_size = 6
num_param = 3
input_param_1 = tf.constant(data, dtype=tf.float32)
input_parameter = expand_2(input_param_1, batch_size, num_param)

# for row in range(batch_size):
#     param_tmp = input_param_1[row,:]
#     param_tmp = tf.reshape(param_tmp, [1, -1])
#     param_mat = tf.matmul(tf.transpose(param_tmp),param_tmp)

#     for i in range(num_param):
#         param_tmp2 = tf.reshape(param_mat[i,i:], [1,-1])
#         #print(param_tmp2.eval(session=sess))
#         param_tmp = tf.concat([param_tmp,param_tmp2], 1)
#     param_tmp = tf.reshape(param_tmp, [-1])

#     try:
#         input_parameter = tf.concat([input_parameter,param_tmp[None, :]],0)
#         input_parameter = tf.reshape(input_parameter, [row+1, -1])
#     except:
#         input_parameter = param_tmp
#         input_parameter = tf.reshape(input_parameter, [row+1, -1])

print(input_parameter.eval(session=sess))
print(tf.shape(input_parameter).eval(session=sess))

Done.
[[  0.   1.   2.   0.   0.   0.   1.   2.   4.]
 [  3.   4.   5.   9.  12.  15.  16.  20.  25.]
 [  6.   7.   8.  36.  42.  48.  49.  56.  64.]
 [  9.  10.  11.  81.  90.  99. 100. 110. 121.]
 [ 12.  13.  14. 144. 156. 168. 169. 182. 196.]
 [ 15.  16.  17. 225. 240. 255. 256. 272. 289.]]
[6 9]
